# Setting up Dataset
A base de dados que iremos usar é uma base de dados sql, portanto tem de se ligar à base de dados com o pacote sqlite3 do python.

In [32]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("soccer.db")

countries = pd.read_sql_query('select * from Country', conn)
countries.head()

leagues = pd.read_sql_query('select * from League', conn)
leagues.head()

matches = pd.read_sql_query('select * from Match', conn)
matches.head()

teams = pd.read_sql_query('select * from Team', conn)
teams.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [40]:
matches = pd.read_sql_query('select home_player_X1 from Match', conn)
matches.loc[~matches["home_player_X1"].isnull()]

,home_player_X1
144,1.0
145,1.0
146,1.0
147,1.0
148,1.0
...,...
25974,1.0
25975,1.0
25976,1.0
25977,1.0


In [23]:
# rascunho
# obter informação sobre jogos especificos de um dado país incluindo o nome
# dos clubes envolvidos e as odds do jogo
df = pd.read_sql_query("select T1.team_long_name, T2.team_long_name, Country.name, date, BSH, BSD, BSA from Country, Match inner join Team as T1 on (Match.home_team_api_id = T1.team_api_id) inner join Team as T2 on (Match.away_team_api_id = T2.team_api_id) where Match.country_id = 17642 and Country.id = Match.country_id;", conn)
df.head()

,team_long_name,team_long_name,name,date,BSH,BSD,BSA
0,FC Porto,CF Os Belenenses,Portugal,2008-08-24 00:00:00,1.33,4.0,9.00
1,Sporting CP,Trofense,Portugal,2008-08-23 00:00:00,1.20,5.0,12.00
2,Vitória Guimarães,Vitória Setúbal,Portugal,2008-08-22 00:00:00,1.91,3.1,3.75
3,FC Paços de Ferreira,SC Braga,Portugal,2008-08-23 00:00:00,2.88,3.0,2.30
4,Amadora,Académica de Coimbra,Portugal,2008-08-24 00:00:00,2.10,3.0,3.25
